# Bike Sharing Prediction Score RMSLE: 0.47290

# EDA
- Analyze the bike charing test vs training data
- Find intracting point to work on

# Pre-process of data
- Change Weather and Season to one-hot column
- Normlized Values to better Predicition

# Analyze data
- Grid search over XG-BOOST to find best paramaters
- Compare XGB vs Linare regression
- Compare number of Regression algoritms

- Select XGB with optimal paramter

# Submit
- get score of 0.47290



In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn import preprocessing

from matplotlib import cm
from matplotlib.ticker import LinearLocator

os.system("pip install fbprophet")
import fbprophet


#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification

In [ ]:
import os
start_path = ""
if os.path.exists('/kaggle/input/bike-sharing-demand'):
    start_path = "/kaggle/input/bike-sharing-demand/"


train_df=pd.read_csv(start_path+'train.csv')
test_df=pd.read_csv(start_path+'test.csv')



seed=1


In [ ]:
from fbprophet import Prophet

def prepare_prophet(df):
    # prepare expected column names
    tmp_df = pd.DataFrame()
    tmp_df['ds']= pd.to_datetime(df['datetime'])
    tmp_df['y'] = np.log(df['count'])
    # define the model
    model = Prophet(yearly_seasonality=True)
    model.add_country_holidays(country_name='US')
    # fit the model
    model.fit(tmp_df)
    return model

def predict_prophet(df,model):
    future = pd.DataFrame()
    future['ds']= pd.to_datetime(df['datetime'])
    print(future.sample())
    # use the model to make a forecast
    forecast = model.predict(future)
    # summarize the forecast
    # print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].sample())
    # print("====")
    print(forecast.sample())
    forecast['exp_yhat']=np.exp(forecast['yhat'])
    df = pd.concat([df,forecast['exp_yhat']],axis=1)
    # plot forecast
    return df

model = prepare_prophet(train_df)
train_df = predict_prophet(train_df,model)
test_df = predict_prophet(test_df,model)



In [ ]:
df = pd.concat([train_df,test_df])

In [ ]:
df.columns.unique()


In [ ]:
df.corr()['count'].drop('count').sort_values()

In [ ]:
df.info()

In [ ]:
test_df.describe()

In [ ]:
train_df.describe()

sesson - [1,2,3,4]
holiday = 0,1 mostly 0
workingday = 0,1 mostly 1
weather = 1,2,3,4 mostly 2-1

temp - [0.82..41] mostly bellow 27
atemp = [0..50] mostly bellow 31

humidity = 0..100 
windspeed = 0..56 mostly bellow 7..16
casual = 0..367 mostly bellow 49

registered = 0..886 mostly bellow 222
count = 1..977 msotly bellow 284


In [ ]:
df.describe()
#the remove change of type to bool made holiday and workday to be removed from describe output

## Idea
- most of the fields can be change into full numbers fields
convery to int instead of float




In [ ]:
plt.rcParams['figure.figsize'] = (18.0, 6.0)
bins = 250
plt.hist(train_df[train_df['season'] == 1]['count'], alpha = 0.1, bins=bins, label='Season-0')
plt.hist(train_df[train_df['season'] == 2]['count'], alpha = 0.2, bins=bins, label='Season-1')
plt.hist(train_df[train_df['season'] == 3]['count'], alpha = 0.3, bins=bins, label='Season-2')
plt.hist(train_df[train_df['season'] == 4]['count'], alpha = 0.4, bins=bins, label='Season-3')

plt.xlabel('bins of days')
plt.ylabel('numbers')
plt.legend(loc='upper right')
plt.xlim(0,150)
plt.grid()
plt.show()

In [ ]:
fig,axes=plt.subplots(4,7)
for i in range(4):
    df = train_df[train_df['season'] == i+1]
    df = df.expanding(min_periods=10).mean()
    axes[i,0].hist(x="temp",data=df,edgecolor="black",linewidth=2,color='red')
    axes[0,0].set_title("temp")
    axes[i,1].hist(x="atemp",data=df,edgecolor="black",linewidth=2,color='red')
    axes[0,1].set_title("atemp")
    axes[i,2].hist(x="windspeed",data=df,edgecolor="black",linewidth=2,color='black')
    axes[0,2].set_title("windspeed")
    axes[i,3].hist(x="humidity",data=df,edgecolor="black",linewidth=2,color='green')
    axes[0,3].set_title("humidity")
    axes[i,4].hist(x="casual",data=df,edgecolor="black",linewidth=2,color='blue')
    axes[0,4].set_title("casual")
    axes[i,5].hist(x="registered",data=df,edgecolor="black",linewidth=2,color='blue')
    axes[0,5].set_title("registered")
    axes[i,6].hist(x="count",data=df,edgecolor="black",linewidth=2,color='blue')
    axes[0,6].set_title("count")
fig.set_size_inches(10,10)



In [ ]:
train_df.plot.scatter(x=["temp"], y=['count'], alpha=0.3, color='green')

In [ ]:


def add_datepart(df, field_name, prefix=None, drop=False, time=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    df[prefix+field_name] = pd.to_datetime(df[field_name], infer_datetime_format=True)
    field = df[prefix+field_name]
    attr = ['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
    'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr:
        if (prefix+n not in df):
            df[prefix + n] = getattr(field.dt, n.lower())
    # Pandas removed `dt.week` in v1.1.10
    if (prefix+"Week" not in df):
        week = field.dt.isocalendar().week if hasattr(field.dt, 'isocalendar') else field.dt.week
        df[prefix+'Week'] = week.astype(np.int32)
    mask = ~field.isna()
#     df[prefix + 'Elapsed'] = np.where(mask,field.values.astype(np.int64) // 10 ** 9,None)
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df

# Convery dates to year,mount,day column
def add_year_day_to_df(test_df):
    test_df["dt_hour"] = [t.hour for t in pd.DatetimeIndex(test_df.datetime)]
#     test_df["day"] = [t.dayofweek for t in pd.DatetimeIndex(test_df.datetime)]
#     test_df["month"] = [t.month for t in pd.DatetimeIndex(test_df.datetime)]
    test_df['year'] = [t.year for t in pd.DatetimeIndex(test_df.datetime)]
    test_df['year'] = test_df['year'].map({2011:0, 2012:1})
    test_df = add_datepart(test_df, 'datetime', prefix="dt_")
    test_df['date_day_idx'] = test_df['year']*365+test_df['dt_Dayofyear']
    test_df['date_day_idx'] -= min(test_df['date_day_idx'])
    test_df['date_hour_idx'] = test_df['date_day_idx']*24+test_df['dt_hour']
#     test_df.drop('datetime',axis=1,inplace=True)
    return test_df


train_df = add_year_day_to_df(train_df)
test_df = add_year_day_to_df(test_df)
df = pd.concat([train_df, test_df])


remove_fields = []
# remove_fields.append('dt_Year')
remove_fields.append('datetime')
remove_fields.append('dt_datetime')
remove_fields.append('casual')
remove_fields.append('registered')
remove_fields.append('count')


train_df.info()

In [ ]:
# test_df['year']*365+test_df['dt_Dayofyear']
df.info()

In [ ]:
fig, axes = plt.subplots(2,2,figsize = (16, 10))
ax11 = plt.subplot(2,2,1)
sns.pointplot(x=train_df['dt_hour'] , y=train_df['count'] , hue = train_df['season'] , ax = ax11)
ax11.set_title('The influence of hour(weekday)')

ax12 = plt.subplot(2,2,2)
sns.pointplot(x=train_df['dt_hour'] , y=train_df['count'] , hue = train_df['weather'] , ax = ax12)
ax12.set_title('The influence of hour(weekday)')

ax2 = plt.subplot(2,2,3)
sns.pointplot(x=train_df['dt_hour'] , y=train_df['count'] , hue = train_df['workingday'] , ax = ax2)
ax2.set_title('The influence of hour(workingday)')

ax3 = plt.subplot(2,2,4)
sns.pointplot(x=train_df['dt_hour'] , y=train_df['count'] , hue = train_df['holiday'] , ax = ax3)
ax3.set_title('The influence of hour(holiday)')

In [ ]:
[df['datetime'].min(),df['datetime'].max()]

In [ ]:
train_df.plot.scatter(x=["dt_Month"], y=['count'], alpha=0.3, color='blue')

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))
train_df.groupby(train_df["dt_hour"])["count"].mean().plot(kind='bar',rot=0,ax=axs)
plt.xlabel("Hour of the day");  # custom x label using matplotlib
plt.ylabel("count");

In [ ]:
#how many uniqe values i got per field
print(df.nunique())
# do we have then a way to translate it into easier to use numbers - normilze 
train_df.value_counts()

## process
remove field from each column and try to predict its values


#process
- use knn or other unsuprvised method to detect groups
- translate those groups as feature



# data
- per month we have max and min sold items
- temp can be convery by *10 and -min values -> will create int


#normlize
- should i normlize per all values or per hour of the or month since at night or day we got diffrent values
- normlize to geosian or uniformal why the diffrent?
- running avrage


# test vs train
what are the diffrenses between the two?


In [ ]:
#change season to one hot
import sklearn.preprocessing
def update_feature_with_convert(df):
    for x in ['season']:
        if (x+'_1' not in df):
            tmp_df = pd.get_dummies(df[x], prefix=x)
            df = pd.concat([df,tmp_df], axis=1)
            if (x in df):
                df = df.drop(x,axis=1)
    return df

train_df = update_feature_with_convert(train_df)
test_df = update_feature_with_convert(test_df)
df = update_feature_with_convert(df)

df.info()



In [ ]:
def remove_redundent_fetures(df):
    for field_name in ['atemp','date_hour_idx']:
        df.drop(field_name, axis=1, inplace=True)
    return df


train_df = remove_redundent_fetures(train_df)
test_df = remove_redundent_fetures(test_df)
df = remove_redundent_fetures(df)




In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from matplotlib.patches import Patch



n_splits=5


# x = train_df.sample(n=1000, weights='count')
print(remove_fields)
X = train_df.drop(remove_fields,axis=1)
y = train_df['count']

print(X.head())
print(X.shape)
print(y.head())
print(y.shape)
time_splits = TimeSeriesSplit(n_splits=n_splits)
index = 1
for train_index, test_index in time_splits.split(X):
    print("train:",[min(train_index),max(train_index)]," test:",[min(test_index),max(test_index)] )




In [ ]:

def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

#Make RLSME Scorer
def rmsle(predicted, actual):
    return np.sqrt(np.square(np.log(predicted + 1) - np.log(actual + 1)).mean())

from sklearn.metrics import make_scorer
scorer= make_scorer(rmsle, greater_is_better=False)


#Train some model on the data
def display_scores(scores):
    print("Scores:", scores)
    print("Score_mean", scores.mean())
    print("Score_std", scores.std())

In [ ]:
#XGBoost hyper-parameter tuning
#https://www.kaggle.com/felipefiorini/xgboost-hyper-parameter-tuning

from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV


def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'reg_alpha' : [0,0.1,1],
        'reg_lambda' : [0,0.1,1],
        'objective': ['reg:squarederror'],
        # 'tree_method': ['gpu_hist']
        'booster' : ['gbtree', 'gblinear', 'dart']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           scoring = scorer,
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = time_splits,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch

# grid_xgb_best = hyperParameterTuning(X, y)
# xgb_best_est = grid_xgb_best.best_estimator_
# grid_xgb_best.best_params_

# best_xgb_param = {'colsample_bytree': 0.7,
#  'learning_rate': 0.01,
#  'max_depth': 7,
#  'min_child_weight': 1,
#  'n_estimators': 500,
#  'objective': 'reg:squarederror',
#  'reg_alpha': 1,
#  'reg_lambda': 1,
#  'subsample': 0.7};

In [ ]:
xgb_best_est = XGBRegressor(
        colsample_bylevel=0.7,
        learning_rate= 0.01,
        max_depth = 7,
        min_child_weight = 1,
        n_estimators = 500,
        objective = 'reg:squarederror',
        reg_alpha = 1,
        reg_lambda = 1,
        subsample = 0.7
    )

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_validate


model_l = {
    'lmodel' : LinearRegression(),
    'xg_boost' : xgb.XGBRegressor(),
    'xg_boost_best' : xgb_best_est,
}
result={}

yy = list(set(y.values))

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, random_state=seed)
for m in model_l:
    print(f"Start {m}")
    lmodel = model_l[m]
    result[m]={}

    
    lmodel.fit(X_train, y_train)
    cv_scores = cross_val_score(lmodel, X, y, cv=time_splits, scoring=scorer)
    print("Cross val:",cv_scores, cv_scores.mean())
    result[m]['mean']=cv_scores.mean()
    idx=0
    for train_index, test_index in time_splits.split(X):
        print("train:",[min(train_index),max(train_index)]," test:",[min(test_index),max(test_index)] ," score:",cv_scores[idx])
        idx+=1

    if (m in ['lmodel']):
        # print the coefficients
        print("itercept:",lmodel.intercept_)
        for x_idx, x_col_name in enumerate(X_train.columns):
            print(f"itercept {x_col_name:20} {round(lmodel.coef_[x_idx],5):10}")
    if (m in ['xg_boost', 'xg_best_est']):
        for x_idx, feature_imp in zip(X_train.columns, lmodel.feature_importances_):
            print(f"itercept {x_idx:20} {round(feature_imp,5):10}")
        
    # for train_index, test_index in splits.split(X):
    #     print("train:",[min(train_index),max(train_index)]," test:",[min(test_index),max(test_index)] )

    lmodel.fit(X_train, y_train)
    pre_train = lmodel.predict(X_train)
    pre_test = lmodel.predict(X_test)
    print("rmsle train:",rmsle(y_train, pre_train))
    result[m]['lrmse_train']=rmsle(y_train, pre_train)
    print("rmsle test:",rmsle(y_test, pre_test))
    result[m]['lrmse']=rmsle(y_test, pre_test)
print(result)



In [ ]:

# Machine learning pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier



In [ ]:
def create_baseline_classifiers(seed=1):
    """Create a list of baseline classifiers.
    
    Parameters
    ----------
    seed: (optional) An integer to set seed for reproducibility

    Returns
    -------
    A list containing tuple of name, model object for each of these algortihms:
    DummyClassifier, LogisticRegression, SGDClassifier, ExtraTreesClassifier, 
    GradientBoostingClassifier, RandomForestClassifier, MultinomialNB, SVC, 
    XGBClassifier.
    
    """
    models = []
    models.append(('dum', DummyClassifier(random_state=seed, strategy='most_frequent')))
    models.append(('log', LogisticRegression(random_state=seed)))
    models.append(('sgd', SGDClassifier(random_state=seed)))
    models.append(('etc', ExtraTreesClassifier(random_state=seed)))
    models.append(('gbm', GradientBoostingClassifier(random_state=seed)))
    models.append(('rfc', RandomForestClassifier(random_state=seed)))
    models.append(('mnb', MultinomialNB()))
    models.append(('svc', SVC(random_state=seed, probability=True)))
    models.append(('xgb', XGBClassifier(seed=seed)))
    return models

def assess_models(X, y, models, cv=5, metrics=['roc_auc', 'f1'],scorer={}, plot=0):
    """Provide summary of cross validation results for models.
    
    Parameters
    ----------
    X: A pandas DataFrame containing feature matrix
    y: A pandas Series containing target vector
    models: A list of models to train
    cv: (optional) An integer to set number of folds in cross-validation
    metrics: (optional) A list of scoring metrics or a string for a metric

    Returns
    -------
    A pandas DataFrame containing summary of baseline models' performance.
    
    """
    summary = pd.DataFrame()
    for name, model in models:
        cross_validate_val = cross_validate(model, X, y, cv=cv, scoring=metrics)
        # print("1 cross_validate_val:\n",cross_validate_val)
        for scorer_name in scorer:
            result1 = cross_val_score(model, X, y, cv=time_splits, scoring=scorer[scorer_name])
            cross_validate_val[scorer_name] = result1
            # print("result1:",result1)
        # print("2 cross_validate_val:\n",cross_validate_val)
        result = pd.DataFrame(cross_validate_val)
        mean = result.mean().rename('{}_mean'.format)
        std = result.std().rename('{}_std'.format)
        min1 = result.min().rename('{}_min'.format)
        max1 = result.max().rename('{}_max'.format)

        summary[name] = pd.concat([mean, std, max1, min1], axis=0)

        #plot real vs predicted values if the plot boolean is true
        if (plot):
            plt.figure(figsize=(5, 5))
            plt.scatter(y_test, y_predicted)
            plt.ylabel("predicted values")
            plt.xlabel("real values")
            #find min and max in the test target values
            minimum = math.ceil(y_test.min())
            maximum = math.ceil(y_test.max())
            #plot a diagonal line accross the scattered plot to better see the difference in values
            #idealy all points should be on the diagonal line
            plt.plot( [minimum,maximum],[minimum,maximum], color='red')


    return summary.sort_index()

def extract_metric(summary, metric):
    """Provide summary of baseline models' performance for a metric.
    
    Parameters
    ----------
    summary: A pandas DataFrame containing the summary of baseline models
    metric: A string specifying the name of the metric to extract info
    
    Returns
    -------
    A pandas DataFrame containing mean, standard deviation, lower and upper
    bound of the baseline models' performance in cross validation according to
    the metric specified.
    
    """
    output = summary[summary.index.str.contains(metric)].T
    output.columns = output.columns.str.replace(f'test_{metric}_', '')
    output.sort_values(by='mean', ascending=False, inplace=True)
    output['lower'] = output['mean'] - 2*output['std']
    output['upper'] = output['mean'] + 2*output['std']
    return output


In [ ]:
# Set target
target = 'count'
features = X.columns

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, random_state=seed)

# Inspect data
print(f"Training data ({X_train.shape[0]} rows): Target distribution")
# print(y_train.value_counts(normalize=True))
print(f"\nTest data ({X_test.shape[0]} rows): Target distribution")
# print(y_train.value_counts(normalize=True))



In [ ]:
# models = create_baseline_classifiers(seed=seed)
# summary = assess_models(X_train, y_train, models, cv=time_splits, metrics=['r2','neg_mean_absolute_error','neg_root_mean_squared_error','neg_mean_squared_log_error'])
# summary

In [ ]:

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor

def create_baseline_regressors(seed=8):
    """Create a list of of baseline regressors.
    
    Parameters
    ----------
    seed: (optional) An integer to set seed for reproducibility

    Returns
    -------
    A list containing tuple of name, model object for each of these algortihms:
    DummyRegressor, LinearRegression, SGDRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, RandomForestRegressor, SVR, XGBRegressor.
    
    """
    models = []
    models.append(('dum', DummyRegressor(strategy='mean')))
    models.append(('ols', LinearRegression()))
    models.append(('sgd', SGDRegressor(random_state=seed)))
    models.append(('etr', ExtraTreesRegressor(random_state=seed)))
    models.append(('gbm', GradientBoostingRegressor(random_state=seed)))
    models.append(('rfr', RandomForestRegressor(random_state=seed)))
    models.append(('svc', SVR()))
    models.append(('xgb', XGBRegressor(seed=seed)))
    models.append(('xgb_best', xgb_best_est))
    return models


models_reg = create_baseline_regressors(seed=seed)
summary_reg = assess_models(X_train, y_train, models_reg, cv=time_splits, metrics=['max_error','r2','neg_mean_absolute_error','neg_median_absolute_error'], scorer={'LRSME':scorer})


In [ ]:
summary_reg

In [ ]:
clf_rf = xgb_best_est
clf_rf.fit(X,y)
pred=clf_rf.predict(test_df.drop(['datetime', 'dt_datetime'],axis=1))

for idx,p in enumerate(pred):
    if (p < 0):
        print(idx,p)
        print(test_df.iloc[idx])
        p=0

d={'datetime':test_df['datetime'],'count':abs(pred)}
ans=pd.DataFrame(d)
ans.to_csv('submission.csv',index=False) # saving to a csv file for predictions.